In [1]:
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import math
import re

In [7]:
def create_sequences_and_targets(encoded_sentences_ids, max_sequence_length=30):
    X_samples = []
    y_samples = []
    step_size = 1
    for sentence_ids in encoded_sentences_ids:
        if len(sentence_ids) < max_sequence_length + 1:
            continue

        # Use sliding window within each sentence
        for i in range(0, len(sentence_ids) - max_sequence_length, step_size):
            input_seq = sentence_ids[i:i+max_sequence_length]
            target_seq = sentence_ids[i+1:i+max_sequence_length+1]

            X_samples.append(input_seq)
            y_samples.append(target_seq)

    # Convert to tensors (no padding needed since all sequences are same length)
    X = torch.tensor(X_samples, dtype=torch.long)
    y = torch.tensor(y_samples, dtype=torch.long)

    return X, y

class NextWordPredictor(nn.Module):
    """
    Neural network for next word prediction with the following architecture:
    Embedding layer -> LSTM layer -> Attention layer -> Fully connected layer

    The model uses Luong (multiplicative) attention to focus on relevant parts
    of the input sequence when making predictions.
    """
    def __init__(self, vocab_size, embeddings_matrix, hidden_dim, num_layers, pad_token_id):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embeddings_matrix.shape[1]
        self.embeddings_matrix = torch.from_numpy(embeddings_matrix).float()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.pad_token_id = pad_token_id

        # Embedding layer: converts token IDs to dense vectors
        self.embedding = nn.Embedding.from_pretrained(self.embeddings_matrix, freeze=False, padding_idx=self.pad_token_id)
        # self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim, padding_idx=self.pad_token_id)

        # LSTM layer: processes sequential information
        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim, self.num_layers, batch_first=True)

        # Attention layer: focuses on relevant parts of the sequence
        self.attention = nn.Linear(self.hidden_dim, 1)

        # Fully connected layer: maps attended features to vocabulary predictions
        self.fc = nn.Linear(self.hidden_dim * 2, self.vocab_size)  # Modified input size to 2 * hidden_dim

        # Dropout for regularization
        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, hidden=None):
        """
        Forward pass through the model.

        Args:
            input_ids: Token IDs (batch_size, seq_len)
            hidden: Initial hidden state for LSTM

        Returns:
            output: Logits for next word prediction (batch_size, seq_len, vocab_size)
            hidden: Final hidden state from LSTM
        """
        batch_size, seq_len = input_ids.size()

        # 1. Embedding layer
        embedded = self.embedding(input_ids)  # (batch_size, seq_len, embedding_dim)
        embedded = self.dropout(embedded)

        # 2. LSTM layer
        lstm_out, hidden = self.lstm(embedded, hidden)  # (batch_size, seq_len, hidden_dim)

        # 3. Attention layer
        # Simpler attention: just weight the LSTM outputs
        attention_weights = torch.softmax(self.attention(lstm_out), dim=1)

        # Weighted average of all positions for each timestep
        attended_output = torch.bmm(attention_weights.transpose(1, 2), lstm_out)

        # Use both attended and original LSTM output
        combined = torch.cat([lstm_out, attended_output.expand_as(lstm_out)], dim=-1)

        # 4. Fully connected layer
        output = self.fc(combined)

        # output = self.fc(context_vector)  # (batch_size, seq_len, vocab_size)

        return output, hidden

    def init_hidden(self, batch_size, device):
        """Initialize hidden state for LSTM."""
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device))

def calculate_accuracy(model, dataloader, device):
    model.eval()
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            batch_size = inputs.size(0)

            hidden = model.init_hidden(batch_size, device)
            output_logits, hidden = model(inputs, hidden)

            predictions = torch.argmax(output_logits, dim=-1)  # (batch_size, seq_len)

            correct_predictions += (predictions == targets).sum().item()
            total_predictions += targets.numel()

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy

def calculate_perplexity(model, dataloader, device):
    model.eval()
    total_loss = 0
    total_tokens = 0
    criterion = nn.CrossEntropyLoss(ignore_index=model.pad_token_id, reduction='sum')

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            batch_size = inputs.size(0)

            hidden = model.init_hidden(batch_size, device)
            output_logits, hidden = model(inputs, hidden)

            loss = criterion(output_logits.view(-1, model.vocab_size), targets.view(-1))

            total_loss += loss.item()
            total_tokens += targets.numel()

    avg_loss = total_loss / total_tokens if total_tokens > 0 else float('inf')
    perplexity = math.exp(avg_loss)

    return perplexity

def train_model(model, train_dataloader, val_dataloader, num_epochs, vocab_size, device="cpu", clip_grad_norm=1.0):
    model.train()
    print("Starting training...")

    optimizer = optim.AdamW(model.parameters(), lr=0.00001)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, steps_per_epoch=len(train_dataloader), epochs=num_epochs, pct_start=0.1)
    # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

    criterion = nn.CrossEntropyLoss(ignore_index=model.pad_token_id)

    for epoch in range(num_epochs):
        total_loss = 0

        for batch_idx, (inputs, targets) in enumerate(train_dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            batch_size = inputs.size(0)
            hidden = model.init_hidden(batch_size, device)
            optimizer.zero_grad()
            output_logits, hidden = model(inputs, hidden)

            hidden = (hidden[0].detach(), hidden[1].detach())
            loss = criterion(output_logits.view(-1, vocab_size), targets.view(-1))
            total_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)

            optimizer.step()

        avg_train_loss = total_loss / len(train_dataloader)

        # --- Validation ---
        model.eval()
        val_loss = 0
        with torch.no_grad():

            for inputs, targets in val_dataloader:
                inputs, targets = inputs.to(device), targets.to(device)

                batch_size = inputs.size(0)
                val_hidden = model.init_hidden(batch_size, device)
                output_logits, val_hidden = model(inputs, val_hidden)
                val_hidden = (val_hidden[0].detach(), val_hidden[1].detach())

                loss = criterion(output_logits.view(-1, vocab_size), targets.view(-1))
                val_loss += loss.item()
        avg_val_loss = val_loss / len(val_dataloader)
        scheduler.step() # Removed epoch argument

        # Calculate accuracy and perplexity every 5 epochs
        if (epoch + 1) % 10 == 0:
            print("Calculating metrics...")
            train_accuracy = calculate_accuracy(model, train_dataloader, device)
            val_accuracy = calculate_accuracy(model, val_dataloader, device)

            train_perplexity = calculate_perplexity(model, train_dataloader, device)
            val_perplexity = calculate_perplexity(model, val_dataloader, device)

            print(f"Epoch [{epoch+1}/{num_epochs}]:")
            print(f"  Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
            print(f"  Train Accuracy: {train_accuracy:.4f}, Val Accuracy: {val_accuracy:.4f}")
            print(f"  Train Perplexity: {train_perplexity:.2f}, Val Perplexity: {val_perplexity:.2f}")
            print("-" * 60)
        else:
            print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

        model.train()

    print("Training finished.")

def evaluate_model(model, test_dataloader, device):
    print("Evaluating model on test set...")

    test_accuracy = calculate_accuracy(model, test_dataloader, device)
    test_perplexity = calculate_perplexity(model, test_dataloader, device)

    print(f"Final Test Results:")
    print(f"  Test Accuracy: {test_accuracy:.4f}")
    print(f"  Test Perplexity: {test_perplexity:.2f}")

    # Check if metrics meet assignment requirements
    print("\nAssignment Requirements Check:")
    print(f"  Test Accuracy > 75%: {'Pass' if test_accuracy > 0.75 else 'Fail'} ({test_accuracy:.1%})")
    print(f"  Perplexity < 250: {'Pass' if test_perplexity < 250 else 'Fail'} ({test_perplexity:.1f})")

    return test_accuracy, test_perplexity

def text_to_indices(text, word_to_idx, max_sequence_length=30):
    words = re.findall(r'\w+|[^\w\s]|\s+', text.lower())
    indices = []

    for word in words:
        if word in word_to_idx:
            indices.append(word_to_idx[word])
        else:
            indices.append(word_to_idx['<UNK>'])

    if max_sequence_length and len(indices) > max_sequence_length:
        indices = indices[-max_sequence_length:]

    return indices

def indices_to_text(indices, idx_to_word):
    words = []
    special_tokens = ['<PAD>', '<UNK>', '<START>', '<END>']
    for idx in indices:
        if idx in idx_to_word:
            if idx_to_word[idx] in special_tokens:
                continue
            words.append(idx_to_word[idx])
    content = ''.join(words)
    content = re.sub(r'\s+', ' ', content).strip()
    return content

def generate_text(model, word_to_idx, idx_to_word, start_text, num_words_to_generate, top5= False, max_sequence_length=30, device="cpu", temperature=1.0):

    model.eval()
    generated_ids = []
    prediction_details = []

    # Encode the start text
    current_input_ids = text_to_indices(start_text, word_to_idx, max_sequence_length)

    generated_ids.extend(current_input_ids)

    # Convert to tensor and add batch dimension
    input_tensor = torch.tensor([current_input_ids], dtype=torch.long).to(device)
    hidden = model.init_hidden(1, device)
    print(f"Generating from: '{start_text}'")

    for i in range(num_words_to_generate):
        with torch.no_grad():
            output_logits, hidden = model(input_tensor, hidden)
            last_word_logits = output_logits[0, -1, :]  # (vocab_size,)
            probabilities = torch.softmax(last_word_logits / temperature, dim=-1)

            top_5_probs, top_5_indices = torch.topk(probabilities, 5)
            top_5_words_and_probs = []
            for j in range(len(top_5_indices)):
                word = idx_to_word.get(top_5_indices[j].item(), '<UNK>')
                prob = top_5_probs[j].item()
                top_5_words_and_probs.append(f"{word} ({prob:.4f})")

            predicted_id = torch.multinomial(probabilities, 1).item()

            prediction_details.append({
                "step": i + 1,
                "top_5_alternatives": top_5_words_and_probs
            })
            generated_ids.append(predicted_id)

            current_input_ids.append(predicted_id)

            if len(current_input_ids) > max_sequence_length:
                current_input_ids = current_input_ids[1:]

            input_tensor = torch.tensor([current_input_ids], dtype=torch.long).to(device)

    generated_text = indices_to_text(generated_ids, idx_to_word)
    if top5 == True:
      print("\n--- Prediction Details ---")
      for detail in prediction_details:
          print(f"Step {detail['step']}:'")
          print(f"  Top 5 likely words: {', '.join(detail['top_5_alternatives'])}")
      print("--------------------------\n")

    return generated_text

In [8]:
with open("embeddings.pkl", 'rb') as f:
    embeddings = pickle.load(f)
with open("data.pkl", 'rb') as f:
    data = pickle.load(f)

train_sequences = data['train_sequences']
val_sequences = data['val_sequences']
test_sequences = data['test_sequences']

word_to_idx = embeddings['word_to_idx']
idx_to_word = embeddings['idx_to_word']
embeddings_matrix = embeddings['embeddings_matrix']

pad_token_id = word_to_idx["<PAD>"]
vocab_size = len(word_to_idx)
print(f"Vocabulary size: {vocab_size}")
embedding_dim = embeddings_matrix.shape[1]
batch_size = 128

X_train, y_train = create_sequences_and_targets(train_sequences)
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

X_val, y_val = create_sequences_and_targets(val_sequences)
val_dataset = TensorDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

X_test, y_test = create_sequences_and_targets(test_sequences)
test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Vocabulary size: 4389


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [10]:
len(X_train)

191178

In [11]:
model = NextWordPredictor(vocab_size, embeddings_matrix, hidden_dim=256, num_layers=2, pad_token_id=pad_token_id)
model.to(device)

NextWordPredictor(
  (embedding): Embedding(4389, 384, padding_idx=0)
  (lstm): LSTM(384, 256, num_layers=2, batch_first=True)
  (attention): Linear(in_features=256, out_features=1, bias=True)
  (fc): Linear(in_features=512, out_features=4389, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [12]:
train_model(model, train_dataloader, val_dataloader, num_epochs=100, vocab_size=vocab_size, device=device)

Starting training...
Epoch [1/100] Train Loss: 4.1094, Validation Loss: 3.3454
Epoch [2/100] Train Loss: 3.1216, Validation Loss: 3.0663
Epoch [3/100] Train Loss: 2.9535, Validation Loss: 2.9205
Epoch [4/100] Train Loss: 2.8096, Validation Loss: 2.8097
Epoch [5/100] Train Loss: 2.7057, Validation Loss: 2.7367
Epoch [6/100] Train Loss: 2.6247, Validation Loss: 2.6690
Epoch [7/100] Train Loss: 2.5422, Validation Loss: 2.5997
Epoch [8/100] Train Loss: 2.4585, Validation Loss: 2.5366
Epoch [9/100] Train Loss: 2.3821, Validation Loss: 2.4822
Calculating metrics...
Epoch [10/100]:
  Train Loss: 2.3118, Val Loss: 2.4302
  Train Accuracy: 0.5871, Val Accuracy: 0.5791
  Train Perplexity: 9.65, Val Perplexity: 11.36
------------------------------------------------------------
Epoch [11/100] Train Loss: 2.2434, Validation Loss: 2.3824
Epoch [12/100] Train Loss: 2.1790, Validation Loss: 2.3385
Epoch [13/100] Train Loss: 2.1191, Validation Loss: 2.2996
Epoch [14/100] Train Loss: 2.0634, Validation 

In [13]:
start_seed_text = "He never spoke of the"
num_words = 10
generated_output = generate_text(model, word_to_idx, idx_to_word, start_seed_text, num_words, top5=True, temperature=0.7, device=device)

print("\n--- Generated Text ---")
print(generated_output)

Generating from: 'He never spoke of the'

--- Prediction Details ---
Step 1:'
  Top 5 likely words:   (1.0000), - (0.0000), , (0.0000), ? (0.0000), ! (0.0000)
Step 2:'
  Top 5 likely words: temple (0.6354), room (0.1881), house (0.0449), men (0.0416), vacancy (0.0189)
Step 3:'
  Top 5 likely words:   (1.0000), . (0.0000), ? (0.0000), - (0.0000), , (0.0000)
Step 4:'
  Top 5 likely words: to (0.9332), for (0.0173), never (0.0150), in (0.0092), as (0.0060)
Step 5:'
  Top 5 likely words:   (0.9987), - (0.0013), ? (0.0000), . (0.0000), , (0.0000)
Step 6:'
  Top 5 likely words: see (0.3064), be (0.1573), leave (0.0850), find (0.0841), do (0.0615)
Step 7:'
  Top 5 likely words:   (0.9972), - (0.0011), ? (0.0008), . (0.0005), , (0.0004)
Step 8:'
  Top 5 likely words: their (0.4301), his (0.1482), such (0.1245), your (0.0876), every (0.0291)
Step 9:'
  Top 5 likely words:   (0.9998), - (0.0002), ? (0.0000), , (0.0000), ! (0.0000)
Step 10:'
  Top 5 likely words: long (0.6300), leave (0.1240), si

In [14]:
start_seed_text = "Boots which extended halfway"
num_words = 50
generated_output = generate_text(model, word_to_idx, idx_to_word, start_seed_text, num_words, temperature=0.7, device=device)

print("\n--- Generated Text ---")
print(generated_output)

Generating from: 'Boots which extended halfway'

--- Generated Text ---
boots which extended halfway up the river in which which you found in which you earn in the opium den, which is unable to which in which


In [15]:
start_seed_text = "I ordered him to pay"
num_words = 50
generated_output = generate_text(model, word_to_idx, idx_to_word, start_seed_text, num_words, temperature=0.7, device=device)

print("\n--- Generated Text ---")
print(generated_output)

Generating from: 'I ordered him to pay'

--- Generated Text ---
i ordered him to pay me and i have got back on i to come back on him on a and passed on either side on either


In [16]:
start_seed_text = "I answered that it had"
num_words = 50
generated_output = generate_text(model, word_to_idx, idx_to_word, start_seed_text, num_words, temperature=0.7, device=device)

print("\n--- Generated Text ---")
print(generated_output)


Generating from: 'I answered that it had'

--- Generated Text ---
i answered that it had been watching it, that it was that there should be safe, it all that it was there, that it must be , well


45